<a href="https://colab.research.google.com/github/S18-Niloy/Greatest_players_gpt/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#read it in
with open("input.txt","r",encoding="utf-8") as f:
  text = f.read()

In [25]:
#length of the text
length = len(text)
print(length)

17354


In [40]:
#checking the first 1000 words
print(text[:1000])

10. George Best

Best’s extraordinary natural talent not only made him one of the most iconic and entertaining players there has been, but the finest from the British Isles.

“If I’d been ugly,” he once said, “You’d never have heard of Pele.” Although that may be a stretch, it was an acknowledgement that, for all of his ability, his womanising stopped him from achieving even more.

It was in 1968 that, aged 22 – his pace, belief, technical ability, balance and creativity in perfect symmetry – Best deservedly followed the other two of United's ‘Holy Trinity’ (Bobby Charlton and Denis Law) in winning the Ballon d’Or.

Best had made his debut for United in 1963 - a boyhood Wolves fan, the Belfast boy was rejected by local club Glentoran for being too small. That didn’t look like a great decision, as the wide man soon began to take British football by storm.

“My best performance was for Northern Ireland against Scotland,” he once told FFT. “We were expected to get slaughtered, but we won 

In [41]:
#all unique  characters
char = sorted(list(set(text)))
len_char = len(char)
print(''.join(char))
print(len_char)


 '(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz£ç–‘’“”…
81


In [42]:
#create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(char)}
itos = {i:ch for i, ch in enumerate(char)}

encode = lambda s:[stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

x = "messi"
print(encode(x))
print(decode(encode('messi')))

[59, 51, 65, 65, 55]
messi


In [54]:
#encode the entire dataset
import torch
data = torch.tensor(encode(text),dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([17354]) torch.int64
tensor([10,  9,  7,  1, 28, 51, 61, 64, 53, 51,  1, 23, 51, 65, 66,  0,  0, 23,
        51, 65, 66, 77, 65,  1, 51, 70, 66, 64, 47, 61, 64, 50, 55, 60, 47, 64,
        71,  1, 60, 47, 66, 67, 64, 47, 58,  1, 66, 47, 58, 51, 60, 66,  1, 60,
        61, 66,  1, 61, 60, 58, 71,  1, 59, 47, 50, 51,  1, 54, 55, 59,  1, 61,
        60, 51,  1, 61, 52,  1, 66, 54, 51,  1, 59, 61, 65, 66,  1, 55, 49, 61,
        60, 55, 49,  1, 47, 60, 50,  1, 51, 60, 66, 51, 64, 66, 47, 55, 60, 55,
        60, 53,  1, 62, 58, 47, 71, 51, 64, 65,  1, 66, 54, 51, 64, 51,  1, 54,
        47, 65,  1, 48, 51, 51, 60,  5,  1, 48, 67, 66,  1, 66, 54, 51,  1, 52,
        55, 60, 51, 65, 66,  1, 52, 64, 61, 59,  1, 66, 54, 51,  1, 23, 64, 55,
        66, 55, 65, 54,  1, 30, 65, 58, 51, 65,  7,  0,  0, 78, 30, 52,  1, 30,
        77, 50,  1, 48, 51, 51, 60,  1, 67, 53, 58, 71,  5, 79,  1, 54, 51,  1,
        61, 60, 49, 51,  1, 65, 47, 55, 50,  5,  1, 78, 45, 61, 67, 77, 50,  1,
        

In [57]:
#splitting data into train and validation
n = int(0.9*len(text))
train_data = data[:n]
validation = data[n:]

In [58]:
block_size = 8
train_data[:block_size+1]

tensor([10,  9,  7,  1, 28, 51, 61, 64, 53])

In [59]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f"when context is:{context} the target:{target} ")

when context is:tensor([10]) the target:9 
when context is:tensor([10,  9]) the target:7 
when context is:tensor([10,  9,  7]) the target:1 
when context is:tensor([10,  9,  7,  1]) the target:28 
when context is:tensor([10,  9,  7,  1, 28]) the target:51 
when context is:tensor([10,  9,  7,  1, 28, 51]) the target:61 
when context is:tensor([10,  9,  7,  1, 28, 51, 61]) the target:64 
when context is:tensor([10,  9,  7,  1, 28, 51, 61, 64]) the target:53 


In [60]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[61, 52,  1, 66, 54, 51,  1, 59],
        [61, 60, 55, 57, 51, 64,  7,  1],
        [55, 51, 64, 65,  1, 34, 61, 64],
        [77, 65,  1, 48, 51, 51, 60,  1]])
targets:
torch.Size([4, 8])
tensor([[52,  1, 66, 54, 51,  1, 59, 61],
        [60, 55, 57, 51, 64,  7,  1, 23],
        [51, 64, 65,  1, 34, 61, 64, 53],
        [65,  1, 48, 51, 51, 60,  1, 50]])
----
when input is [61] the target: 52
when input is [61, 52] the target: 1
when input is [61, 52, 1] the target: 66
when input is [61, 52, 1, 66] the target: 54
when input is [61, 52, 1, 66, 54] the target: 51
when input is [61, 52, 1, 66, 54, 51] the target: 1
when input is [61, 52, 1, 66, 54, 51, 1] the target: 59
when input is [61, 52, 1, 66, 54, 51, 1, 59] the target: 61
when input is [61] the target: 60
when input is [61, 60] the target: 55
when input is [61, 60, 55] the target: 57
when input is [61, 60, 55, 57] the target: 51
when input is [61, 60, 55, 57, 51] the target: 64
when input is [61,

In [61]:
print(xb) # our input to the transformer

tensor([[61, 52,  1, 66, 54, 51,  1, 59],
        [61, 60, 55, 57, 51, 64,  7,  1],
        [55, 51, 64, 65,  1, 34, 61, 64],
        [77, 65,  1, 48, 51, 51, 60,  1]])


In [63]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(len_char)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 81])
tensor(5.0899, grad_fn=<NllLossBackward0>)

…S0C–DS0-Pdvl4çd:”-(‘
6.'“…w:;76RZ67?E JwWoDt7EsOZgd ;CP6etb
uo7c–,-.GU.G8‘6vHneS/IkPdURILqETe1‘An85


In [65]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(len_char)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 81])
tensor(5.0899, grad_fn=<NllLossBackward0>)

…S0C–DS0-Pdvl4çd:”-(‘
6.'“…w:;76RZ67?E JwWoDt7EsOZgd ;CP6etb
uo7c–,-.GU.G8‘6vHneS/IkPdURILqETe1‘An85


In [66]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [70]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

4.62875509262085


In [68]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


–oBvLWUS:gGvupS£YbqoR;6R; q9Ir)C-dq(’W-UDY(…?Bz-'mem0gN;
pNQQ£m8F(D0P£mzm8tV£mn£We2kh
…
b’OU9Gc:gZNA.RDT0.ji7“Pd(U/…3EreW9gjB:6vNPBt7t(7;8B.xQ3Rkf7Mç?m”85RB)5E67t
dc£ZçChC?““8q
URGGgdOCpV0jB'Bh“’R)C/8Gg-ZxN…vUekV;zW1
a/“5:B8ZçU6Uc9’tn’Lw,78SnkE9,BksCHh…':G-MVuPZç?Ig;h-Ff)‘
hNV.p0EsGU£s”,‘Qri’QT-lJJJyh0Yo8/8z:gyF£W/8y9G/”O c:jiM)
W/8S0/8MudCLdq5 GTE;xOa.mBegB;dçz)ç)e3c1bZoYSzov5£Iy1v,KO.w(6G1w)Q.oNN,fV’ILqoczpoc?eO'k4çYY.'FLW)R.
z‘ix3Elp'h6R’eq1B/6nFV0Ho(ç2zY,R;nZaJ9k4RBaIfQ3,VODk8A4)N–f4
htOU4A5


In [71]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [72]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [73]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

In [74]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [75]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


True

In [76]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [77]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [78]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [79]:
k.var()

tensor(1.0449)

In [80]:
q.var()

tensor(1.0700)

In [81]:
wei.var()

tensor(1.0918)

In [82]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [83]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [84]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [85]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [86]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [87]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.211793 M parameters
step 0: train loss 4.4953, val loss 4.4803
step 100: train loss 2.6757, val loss 2.6205
step 200: train loss 2.5163, val loss 2.5193
step 300: train loss 2.4242, val loss 2.4397
step 400: train loss 2.3612, val loss 2.3730
step 500: train loss 2.2570, val loss 2.2888
step 600: train loss 2.1629, val loss 2.2440
step 700: train loss 2.0810, val loss 2.2104
step 800: train loss 1.9947, val loss 2.1697
step 900: train loss 1.9149, val loss 2.1436
step 1000: train loss 1.8229, val loss 2.1327
step 1100: train loss 1.7407, val loss 2.0937
step 1200: train loss 1.6689, val loss 2.1158
step 1300: train loss 1.5776, val loss 2.1173
step 1400: train loss 1.5113, val loss 2.1370
step 1500: train loss 1.4444, val loss 2.1304
step 1600: train loss 1.3759, val loss 2.1770
step 1700: train loss 1.3117, val loss 2.1965
step 1800: train loss 1.2302, val loss 2.2559
step 1900: train loss 1.1813, val loss 2.2697
step 2000: train loss 1.1177, val loss 2.3218
step 2100: train loss 1.